# GPT2 - PRETRAINED

---

In [1]:
!pip install -q datasets peft bitsandbytes evaluate accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.8 MB/s eta 0:00:00


In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


---

# Khai báo tv và thiết lập các tham số chung

In [3]:
import os, re, math, json, torch, random
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import evaluate

In [4]:
device = torch.device("cuda")

MODEL_NAME="gpt2"
#True-> QLoRA
USE_4BIT = False
OUTPUT_DIR= "ft-gpt"
#cắt tối đa mỗi mẫu
MAX_LEN=256

# 4 x 4 hoặc 8 x 1, nhưng xài 16 x 2 cho nhanh -> Thực tế là 32
BATCH_SIZE= 16
GRAD_ACCUM= 2

N_EPOCHS= 2
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

---

# Tiền xử lý DL

In [5]:
#Tiền xử lí, lấy dataset
def preprocess(ds, max_samples=None):
    q,a = [], []
    for ex in ds:
        t = ex["text"]
        if "<human>:" in t and "<bot>:" in t:
            _,_,hum = t.partition("<human>:")
            hum,_,bot = hum.partition("<bot>:")
            q.append(hum.strip())
            a.append(re.sub(r"(?i)LƯU Ý.*$", "", bot).strip())
            if max_samples and len(q)>=max_samples:
                break
    return q,a

raw = load_dataset("RafaelMPereira/HealthCareMagic-100k-Chat-Format-en")["train"]

# max_samples=None
ins, outs = preprocess(raw, max_samples=None)

tr_i, te_i, tr_o, te_o = train_test_split(ins, outs, test_size=0.1, random_state=SEED)
tr_i, va_i, tr_o, va_o = train_test_split(tr_i, tr_o, test_size=0.1111, random_state=SEED)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

HealthCareMagic-100k-en.jsonl:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

In [6]:
print(len(ins))

112165


# BFE token hoá

In [7]:
#TOKENIZER BPE, GPT-2 dùng byte-level BPE
tok=AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

#GPT-2 không có token pad mặc định
if tok.pad_token is None:
    tok.add_special_tokens({"pad_token":"<pad>"})

if USE_4BIT:
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    base = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_cfg)
    base=prepare_model_for_kbit_training(base)
else:
    base=AutoModelForCausalLM.from_pretrained(MODEL_NAME).to("cuda")

#LoRA (không cần với GPT-2 nhỏ, rất hữu ích với lớn hơn 1 B tham số)
if MODEL_NAME.lower()!="gpt2":
    lora_cfg = LoraConfig(
        task_type="CAUSAL_LM",
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules=["q_proj", "v_proj"]
    )
    model=get_peft_model(base, lora_cfg)
else:
    model=base

model.resize_token_embeddings(len(tok))

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50258, 768)

# Lớp DATASET CHO GPT

In [ ]:
class GPTDataset(torch.utils.data.Dataset):
    def __init__(self, qs, ans, tokenizer, max_len=256):
        self.q, self.a, self.tok, self.max = qs, ans, tokenizer, max_len

    def __len__(self):
      return len(self.q)

    def __getitem__(self, i):
        prompt=f"Patient: {self.q[i]}\nDoctor:"
        text=prompt + " " + self.a[i] + self.tok.eos_token
        enc=self.tok(text, truncation=True, max_length=self.max, return_tensors="pt", padding="max_length")
        labels = enc.input_ids.clone()
        #mask phần prompt (không tính loss)
        prompt_enc = self.tok(prompt, truncation=True, max_length=self.max, return_tensors="pt", padding="max_length")
        labels[0, :len(prompt_enc["input_ids"][0][prompt_enc["attention_mask"][0] == 1])] = -100

        return {"input_ids": enc.input_ids[0],
                "attention_mask": enc.attention_mask[0],
                "labels": labels[0],
                "original_question": self.q[i]}

train_ds=GPTDataset(tr_i, tr_o, tok, MAX_LEN)
val_ds= GPTDataset(va_i, va_o, tok, MAX_LEN)
test_ds=GPTDataset(te_i, te_o, tok, MAX_LEN)
collator=DataCollatorForLanguageModeling(tok, mlm=False)

training_args = TrainingArguments(
    OUTPUT_DIR,
    num_train_epochs=N_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=5e-5,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    fp16=(not USE_4BIT),
    report_to="none",
    seed=SEED
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=collator
)
trainer.train()
model.save_pretrained(f"{OUTPUT_DIR}/best")
tok.save_pretrained(f"{OUTPUT_DIR}/best")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.891500,2.851763


('ft-gpt/best/tokenizer_config.json',
 'ft-gpt/best/special_tokens_map.json',
 'ft-gpt/best/vocab.json',
 'ft-gpt/best/merges.txt',
 'ft-gpt/best/added_tokens.json',
 'ft-gpt/best/tokenizer.json')

In [ ]:
#trainer.train()

In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b6b2edf8a2451474b765b6e05b92ae4c7259b27826ead4872ebb4b984c4557e8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
trainer.train(resume_from_checkpoint="ft-gpt2/checkpoint-5608")

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=5608, training_loss=0.0, metrics={'train_runtime': 0.0018, 'train_samples_per_second': 100672271.373, 'train_steps_per_second': 3145868.24, 'total_flos': 2.3441553358848e+16, 'train_loss': 0.0, 'epoch': 1.9994651453021928})

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Load tokenizer
tok=AutoTokenizer.from_pretrained("ft-gpt2/checkpoint-5608", use_fast=True)

#đảm bảo có pad token
if tok.pad_token is None:
    tok.add_special_tokens({"pad_token":"<pad>"})

#load lại model
base=AutoModelForCausalLM.from_pretrained("gpt2")
base=base.to(device)

model=AutoModelForCausalLM.from_pretrained("ft-gpt2/checkpoint-5608").to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

# Test

In [15]:
# import os
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from google.colab import drive
# drive.mount('/content/drive', force_remount=False)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dst_folder = '/content/drive/MyDrive/LNP_OUT/pretrained_checkpoint-5608'

# tok = AutoTokenizer.from_pretrained(
#     dst_folder,
#     use_fast=True,
#     local_files_only=True
# )
# if tok.pad_token is None:
#     tok.add_special_tokens({"pad_token": "<pad>"})

# model = AutoModelForCausalLM.from_pretrained(
#     dst_folder,
#     local_files_only=True
# ).to(device)

# model.resize_token_embeddings(len(tok))
# model.eval()

# print("Loaded GPT-2 fine-tuned model from Drive successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded GPT-2 fine-tuned model from Drive successfully!


In [21]:
#Hàm sinh câu trả lời
@torch.no_grad()
def gpt_reply(question, max_new=128, top_p=0.9, temperature=0.7):
  # ''''
  # sampling đa dạng
  #     model không cứng nhắc chọn token xác suất cao nhất (như greedy),
  #     mà sampling trong top 90% xác suất, điều chỉnh độ “sáng tạo” với temperature.
  #     =>Nhờ vậy câu trả lời bớt lặp, phong phú hơn.

  #  '''
    model.eval()
    prompt = f"Patient: {question}\nDoctor:"
    inp = tok(prompt, return_tensors="pt").to(model.device)
    out = model.generate(
        **inp, max_new_tokens=max_new,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tok.pad_token_id, eos_token_id=tok.eos_token_id
    )
    gen = tok.decode(out[0], skip_special_tokens=True)
    if gen.startswith(prompt):
        return gen[len(prompt):].strip()
    else:
        return gen.strip()

In [25]:
#Đánh giá Log-Perplexity, BLEU-4, ROUGE-L
bleu  = evaluate.load("bleu")
rouge = evaluate.load("rouge")

refs, hyps = [], []
nll, n_tok = 0.0, 0

model.eval()
for ex in test_ds:
    ids    = ex["input_ids"].unsqueeze(0).to(model.device)
    labels = ex["labels"].unsqueeze(0).to(model.device)
    #lấy câu hỏi gốc
    original_question = ex["original_question"]

    with torch.no_grad():
        out = model(input_ids=ids, labels=labels)
        if not torch.isnan(out.loss):
             nll += out.loss.item() * (labels != -100).sum().item()
    n_tok += (labels != -100).sum().item()

    ref_txt = tok.decode(labels[0][labels[0] != -100], skip_special_tokens=True)
    hyp_txt = gpt_reply(original_question)
    refs.append([ref_txt]); hyps.append(hyp_txt)

log_ppl = nll / n_tok if n_tok > 0 else float('inf')
ppl = math.exp(log_ppl)

print(f"Log-Perplexity: {log_ppl:.4f} |  PPL: {ppl:.2f}")
print(f"BLEU-4       : {bleu.compute(predictions=hyps, references=refs)['bleu']*100:.2f}")
print(f"ROUGE-L      : {rouge.compute(predictions=hyps, references=[r[0] for r in refs])['rougeL']*100:.2f}")

Log-PPL: 5.3660 | PPL: 214.00
BLEU-4 : 5.36
ROUGE-L: 15.75


In [22]:
#25 mẫu Q-A để human-rating
import textwrap, datetime
N = 25
N = min(N, len(te_i))
idxs = random.sample(range(len(te_i)), N)
fn = os.path.join(OUTPUT_DIR, f"human_eval_{datetime.date.today()}.txt")
with open(fn, "w", encoding="utf-8") as fp:
    for k, idx in enumerate(idxs, 1):
        q = te_i[idx]; ref = te_o[idx]
        hyp = gpt_reply(q)
        fp.write(f"{'='*78}\n[{k:02d}] QUESTION:\n{textwrap.fill(q,70)}\n\n")
        fp.write(f"REFERENCE ANSWER:\n{textwrap.fill(ref,70)}\n\n")
        fp.write(f"MODEL ANSWER:\n{textwrap.fill(hyp,70)}\n\n")
print(f"Đã ghi {N} mẫu vào {fn}")

Đã ghi 25 mẫu vào ft-gpt/human_eval_2025-05-21.txt


In [ ]:
from google.colab import drive
import shutil, os

#Mount Google Drive
drive.mount('/content/drive', force_remount=True)
src_folder = '/content/ft-gpt2/checkpoint-5608'
dst_folder = '/content/drive/MyDrive/LNP_OUT/checkpoint-5608'
os.makedirs(os.path.dirname(dst_folder), exist_ok=True)
#COPy lại thư mục
shutil.copytree(src_folder, dst_folder)

Mounted at /content/drive
